In [18]:
import tensorflow
import numpy as np
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D

In [19]:
image_size = 224


In [20]:
def getModel_vgg16(image_size):
    ## VGG16 architecture: https://arxiv.org/pdf/1409.1556.pdf
    ## Also may be able to use VGG-Face pre-trained weights. Not sure if the VGG-Face
    ## architecture is the same as the VGG-16. 
    ## Note: if the architectures don't match, they probably only differ by a small 
    ## amount, so we can probably create a separate VGG-Face model based on our VGG16
    ## and then use the weights from Oxford: http://www.robots.ox.ac.uk/~vgg/software/vgg_face/ 
    
    # This implementation is based on Configuration D from page 3 of 1409.1556.pdf, so 16 weight layers total: 
    
    # Input (image): 
    # Note, I read somewhere that for tensorflow the order matters for performance, so check if this should be (1, img_size, img_size) instead? 
    image_input = Input(shape=(image_size, image_size, 1))
    
    # Group 1: 
    x = Conv2D(filters=64, kernel_size=(3, 3) strides=(1,1), activation="relu")(image_input)
    x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    # Group 2: 
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        
    # Group 3: 
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        
    # Group 4: 
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
        
    # Group 5:
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1,1), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    # Final weight layers:
    x = Dense(units=4096)(x)
    x = Dense(units=4096)(x)
    
    # Original implementation has the following, but we don't want these here, since our task is different: 
    #Dense(units=1000)(x)
    # Softmax()
    
    return x
    pass


In [21]:
model = getModel_vgg16(image_size)


TypeError: __init__() missing 2 required positional arguments: 'filters' and 'kernel_size'